# CUDA Convolution Accelerator - Colab Setup\n\nThis notebook sets up the CUDA convolution project in Google Colab.\n\n## Prerequisites\n\n**IMPORTANT:** Set runtime to GPU!\n- Runtime → Change runtime type → Hardware accelerator → GPU\n- Recommended: T4 or better

## Step 1: Verify GPU Access

In [ ]:
# Check NVIDIA GPU\n!nvidia-smi\n\nimport torch\nif torch.cuda.is_available():\n    print(f"\n✓ GPU Available: {torch.cuda.get_device_name(0)}")\n    print(f"✓ CUDA Version: {torch.version.cuda}")\nelse:\n    print("\n✗ No GPU detected. Please change runtime type to GPU.")

## Step 2: Install Dependencies

In [ ]:
# Install CuPy and other dependencies\n!pip install cupy-cuda11x scipy matplotlib pillow tqdm pytest -q\n\nprint("✓ Dependencies installed")

## Step 3: Clone Repository\n\nChoose one of the following options:

In [ ]:
# Option A: Clone from GitHub (replace with your repo URL)\n!git clone https://github.com/elcruzo/cuda-conv.git\n%cd cuda-conv

In [ ]:
# Option B: Upload files manually\nfrom google.colab import files\nimport zipfile\n\n# Upload your project zip file\nuploaded = files.upload()\n\n# Extract\nfor filename in uploaded.keys():\n    if filename.endswith('.zip'):\n        with zipfile.ZipFile(filename, 'r') as zip_ref:\n            zip_ref.extractall('.')\n        print(f"✓ Extracted {filename}")

## Step 4: Verify Installation

In [ ]:
# Test imports\nimport cupy as cp\nfrom src.api import convolve\nfrom src.presets import get_kernel, list_kernels\n\nprint(f"✓ CuPy version: {cp.__version__}")\nprint(f"✓ GPU: {cp.cuda.Device().name.decode()}")\nprint(f"✓ Available kernels: {', '.join(list_kernels())}")\n\n# Quick test\nimport numpy as np\ntest_img = np.random.rand(32, 32).astype(np.float32)\ntest_kernel = get_kernel('box_blur')\nresult = convolve(test_img, test_kernel)\n\nprint(f"\n✓ Test convolution successful! Result shape: {result.shape}")

## Step 5: Generate Sample Images (if needed)

In [ ]:
# Generate sample images\n!python3 scripts/generate_sample_images.py\n\nprint("✓ Sample images generated")

## Step 6: Run Tests

In [ ]:
# Run test suite\n!pytest tests/ -v\n\nprint("\n✓ All tests completed")

## Next Steps\n\nNow you're ready to use the CUDA convolution accelerator!\n\nTry:\n- Open `notebooks/01_demo_speed.ipynb` for benchmarks\n- Open `notebooks/02_examples.ipynb` for visual examples\n- Use the CLI: `!python scripts/cli.py --help`